# MAML

## Index
1. Import Library  
2. Path, Parameter, Hyper Parameter Setting  
3. Custom Data Generator  

# 1. Import Library

In [1]:
import tensorflow as tf
import numpy as np
import keras
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import scipy

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import layers, losses, metrics, optimizers
from keras.optimizers import Adam

import random

from tqdm import tqdm_notebook

# 2. Path, Parameter, Hyper Parameter Setting

In [2]:
'''

'''

train_path = 'cifar10/train'
test_path = 'cifar10/test'

n_way_size = 5
k_shot_size = n_way_size * 3

# MAML Training HyperParameter Setting
learning_rate_outer = 0.001
num_total_tasks = 100
num_inner_updates = 15
learning_rate_inner = 0.001

# 3. Custom Data Generator

In [3]:
class CustomGenerator():
    def __init__(self, train_path: str, test_path: str, n_way_size: int, k_shot_size: int):
        self.train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)
        self.test_datagen = ImageDataGenerator(rescale=1./255)
        
        self.train_path = train_path
        self.test_path = test_path
        
        self.original_class_list = sorted(os.listdir(train_path))
        
        self.n_way_size = n_way_size
        self.k_shot_size = k_shot_size
        
        
    def select_random_n_way(self):
        result = [element for element in random.sample(self.original_class_list, self.n_way_size)]
        return result
    
    def custom_generator(self):
        
        random_n_way = self.select_random_n_way()
        
        supportset_generator = self.train_datagen.flow_from_directory(self.train_path,
                                                            classes=sorted(random_n_way),
                                                            batch_size = self.k_shot_size,
                                                            target_size = (256, 256),
                                                            subset="training",
                                                            class_mode='categorical', 
                                                            shuffle=True)


        queryset_generator = self.test_datagen.flow_from_directory(self.test_path, 
                                                          classes=sorted(random_n_way),
                                                          batch_size = self.n_way_size,
                                                          target_size = (256, 256),
                                                          class_mode='categorical', 
                                                          shuffle=True)
        
        print(f"selected random classes: {random_n_way}")
        
        return supportset_generator, queryset_generator
    

    
    
# =============================class method test=================================
test = CustomGenerator(train_path=train_path, test_path=test_path, n_way_size=n_way_size, k_shot_size=k_shot_size)
supportset_generator, queryset_generator = test.custom_generator()
print()

print("check shape of data shape")
for x_data, class_data in supportset_generator:
    print(f"input data shape from train_generator: {x_data.shape}")
    print(f"class data shape from train_generator: {class_data.shape}")
    break

for x_data, class_data in queryset_generator:
    print(f"input data shape from test_generator: {x_data.shape}")
    print(f"class data shape from test_generator: {class_data.shape}")
    break    

Found 22500 images belonging to 5 classes.
Found 5000 images belonging to 5 classes.
selected random classes: ['dog', 'truck', 'cat', 'horse', 'bird']

check shape of data shape
input data shape from train_generator: (15, 256, 256, 3)
class data shape from train_generator: (15, 5)
input data shape from test_generator: (5, 256, 256, 3)
class data shape from test_generator: (5, 5)


In [4]:
class AlexNet():
    def __init__(self):
        self.model = self.model_architecture()
        
    def model_architecture(self):
        
        model = keras.Sequential([
            layers.Conv2D(96, (11, 11), strides=(4, 4), padding='valid', activation='relu', input_shape=(256, 256, 3)),
            layers.BatchNormalization(),
            layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
            layers.Conv2D(256, (5, 5), strides=(1, 1), padding='same', activation='relu'),
            layers.BatchNormalization(),
            layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
            layers.Conv2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu'),
            layers.Conv2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu'),
            layers.Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu'),
            layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
            layers.Flatten(),
            layers.Dense(512, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(0.5),
            layers.Dense(256, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(0.5),
            layers.Dense(128, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(0.5),
            # layers.Dense(10, activation='softmax')
            layers.Dense(5, activation='softmax')  # 클래스 개수에 맞게 변경
        ])
        
        return model
        

In [5]:
class MAML():
    def __init__(self, n_way_size, k_shot_size, learning_rate_outer, num_total_tasks, num_inner_updates, learning_rate_inner):
        self.n_way_size = n_way_size
        self.k_shot_size = k_shot_size
        self.learning_rate_outer = learning_rate_outer
        self.num_total_tasks = num_total_tasks
        self.num_inner_updates = num_inner_updates
        self.learning_rate_inner = learning_rate_inner
        
    
    def base_model_load(self):
        # Base 모델 구성
        # 다른 파일에서 불러오는 것으로 변경하기
        model = AlexNet()
        model = model.model
        return model
    
    def base_model_compile(self, inner_or_outer):
        model_clone = keras.models.clone_model(self.base_model_load())
        model_clone.compile(loss='categorical_crossentropy', 
                           optimizer = Adam(lr=inner_or_outer),
                           metrics=['accuracy'])
        
        return model_clone
    
    
    def maml_training(self):
        
        # Initialize the maml model's parameters.
        ## maml optimizer 및 loss function check
        
        maml_model = self.base_model_compile(inner_or_outer=self.learning_rate_outer)
        
        # Repeat for a specified number of outer loop updates
        # outer optimization: 여러 tasks에서 meta learner가 학습하는 과정
            

        # Allocate task
        for task in range(self.num_total_tasks):
            print(task)
            # base model clone for each task inner learning
            inner_clone_model = self.base_model_compile(inner_or_outer=self.learning_rate_inner)
            inner_clone_model.set_weights(maml_model.get_weights())

            # Load support set data, query set data generator for specified task
            data_generator = CustomGenerator(train_path=train_path, test_path=test_path, n_way_size=self.n_way_size, k_shot_size=self.k_shot_size)
            supportset_generator, queryset_generator = data_generator.custom_generator()

            support_set_data, support_set_labels = supportset_generator.next()
            query_set_data, query_set_labels = queryset_generator.next()
            
            for inner_step in tqdm_notebook(range(num_inner_updates)):
                with tf.GradientTape() as tape:

                    predictions = inner_clone_model(support_set_data, training=True)
                    loss = tf.reduce_mean(tf.square(predictions - support_set_labels))
                    gradients = tape.gradient(loss, inner_clone_model.trainable_variables)
                    inner_clone_model.optimizer.apply_gradients(zip(gradients, inner_clone_model.trainable_variables))



            # Compute the adaptation loss on an query set for the current task
            with tf.GradientTape() as tape:
                support_set_data, support_set_labels = supportset_generator.next()
                query_set_data, query_set_labels = queryset_generator.next()

                adaption_loss = tf.reduce_mean(tf.square(inner_clone_model(query_set_data, training=False) - query_set_labels))
            
            print(adaption_loss)
            # Update the model parameters in the outer loop based on adaptation loss

            outer_gradients = tape.gradient(adaption_loss, maml_model.trainable_variables)
            filtered_gradients = [grad for grad in outer_gradients if grad is not None]
            maml_model.optimizer.apply_gradients(zip([-self.learning_rate_outer * grad for grad in filtered_gradients], maml_model.trainable_variables))

                       
        return maml_model
                   

In [6]:
maml = MAML(n_way_size, k_shot_size, learning_rate_outer, num_total_tasks, num_inner_updates, learning_rate_inner)

In [7]:
result = maml.maml_training()

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


0
Found 22500 images belonging to 5 classes.
Found 5000 images belonging to 5 classes.
selected random classes: ['cat', 'horse', 'deer', 'ship', 'automobile']


<ipython-input-5-47406db813f5>:52: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inner_step in tqdm_notebook(range(num_inner_updates)):


  0%|          | 0/15 [00:00<?, ?it/s]

tf.Tensor(0.31022322, shape=(), dtype=float32)
1
Found 22500 images belonging to 5 classes.
Found 5000 images belonging to 5 classes.
selected random classes: ['truck', 'automobile', 'airplane', 'ship', 'cat']


...

tf.Tensor(0.32173634, shape=(), dtype=float32)
98
Found 22500 images belonging to 5 classes.
Found 5000 images belonging to 5 classes.
selected random classes: ['frog', 'airplane', 'ship', 'bird', 'deer']


  0%|          | 0/15 [00:00<?, ?it/s]

tf.Tensor(0.3682383, shape=(), dtype=float32)
99
Found 22500 images belonging to 5 classes.
Found 5000 images belonging to 5 classes.
selected random classes: ['frog', 'deer', 'dog', 'cat', 'airplane']


  0%|          | 0/15 [00:00<?, ?it/s]

tf.Tensor(0.23999904, shape=(), dtype=float32)


In [8]:
scores = result.evaluate_generator(queryset_generator)

<ipython-input-8-3b5e7ab15b6b>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = result.evaluate_generator(queryset_generator)


In [9]:
print(scores[1]*100)

20.000000298023224
